# Data Wrangling

This notebook presents different data wrangling techniques used commonly

In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

from IPython.display import display

pd.options.mode.chained_assignment = None

## Utilities

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """
    
    # sentinels
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

## Generate a Sample Dataset

In [3]:
df = generate_sample_data(row_count=1000)

### Describe the Dataset

In [4]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 60


Sample Indices with missing data:: [0, 1, 3, 4, 5]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 970 non-null float64


d:\users\703177402\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:98: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
d:\users\703177402\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,2016-01-01,-101,0,16,NaN,n
1,1001,NaN,5614,162,23,4699.26,n
2,1002,2016-01-13,5010,873,4,240.05,n
3,1003,NaN,5884,408,26,2754.57,n
4,1004,NaN,5015,774,19,1931.35,n


### Rename Columns

In [5]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [6]:
cleanup_column_names(df)

In [7]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


### Sort Rows on defined attributes

In [8]:
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
704,-1,2016-05-02,5015,774,16,5245.44,c
95,-1,2016-01-22,5594,184,11,4399.92,a
580,-1,2016-01-20,5975,750,12,4380.50,c
496,-1,2016-01-01,5761,1031,38,4361.07,b
70,-1,NaN,5785,710,31,3789.10,d


### Rearrange Columns in a Dataframe

In [9]:
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,2016-01-01,-101,n,0,16,NaN
1,1001,NaN,5614,n,162,23,4699.26
2,1002,2016-01-13,5010,n,873,4,240.05
3,1003,NaN,5884,n,408,26,2754.57
4,1004,NaN,5015,n,774,19,1931.35


### Filtering Columns

Using Column Index

In [10]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[   0  162  873  408  774  730  472  500 1017  273]


Using Column Name

In [11]:
# print 10 values of quantity purchased
print(df.quantity_purchased.values[0:10])

[16 23  4 26 19  7 32 28 24 28]


Using Column Datatype

In [12]:
# print 10 values of columns with data type float
print(df.select_dtypes(include=['float64']).values[:10,0])

[    nan 4699.26  240.05 2754.57 1931.35 3276.87 3445.96 1066.43   16.25
 3647.24]


### Filtering Rows

Select specific rows

In [13]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-06-01,5734,266,23,126.20,n
501,1501,2016-11-02,5614,162,4,3066.07,a
20,1020,NaN,5792,658,11,4150.16,n


Exclude Specific Row indices

In [14]:
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,NaN,5614,162,23,4699.26,n
2,1002,2016-01-13,5010,873,4,240.05,n
3,1003,NaN,5884,408,26,2754.57,n
4,1004,NaN,5015,774,19,1931.35,n
5,1005,NaN,5535,730,7,3276.87,n


Conditional Filtering

In [15]:
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
3,1003,NaN,5884,408,26,2754.57,n
6,1006,2016-01-16,5173,472,32,3445.96,n
7,1007,NaN,5668,500,28,1066.43,n
9,1009,NaN,5687,273,28,3647.24,n
11,1011,NaN,5106,881,30,861.17,n


Offset from top of the dataframe

In [16]:
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-01-22,5443,1058,9,5472.41,b
101,1101,2016-01-22,5614,162,15,2065.63,d
102,1102,2016-05-02,5010,873,9,73.02,c
103,1103,2016-04-02,5884,408,34,4785.90,b
104,1104,2016-01-13,5015,774,9,1847.31,d


Offset from bottom of the dataframe

In [17]:
display(df[-10:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
990,1990,2016-06-02,5709,582,27,3236.80,c
991,1991,2016-11-01,5012,995,41,2199.65,d
992,1992,2016-07-02,5542,1098,34,3061.29,b
993,1993,2016-06-02,5681,327,10,550.62,c
994,1994,2016-05-01,5910,947,35,5132.45,d


### TypeCasting/Data Type Conversion

In [18]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int32
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int32
price                        float64
user_type                     object
dtype: object


### Apply/Map Usage

Map : Create a derived attribute using map

In [19]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-07-02,5594,184,11,2538.75,b,new
996,1996,2016-01-24,5761,1031,14,791.87,b,new
997,1997,2016-01-26,5483,1086,7,435.93,a,new
998,1998,2016-01-23,5253,676,39,307.67,b,new
999,1999,2016-10-01,5362,654,24,NaN,d,loyal_existing


Apply: Using apply to get attribute ranges

In [20]:
display(df.select_dtypes(include=[np.number]).apply(lambda x: 
                                                        x.max()- x.min()))

serial_no             2000.00
user_id               6095.00
product_id            1098.00
quantity_purchased      40.00
price                 8635.93
dtype: float64

Applymap: Extract week from date

In [21]:
df['purchase_week'] = df[['date']].applymap(lambda dt:dt.week 
                                                if not pd.isnull(dt.week) 
                                                else 0)

In [22]:
display(df.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-01,-101,0,16,NaN,n,error,53
1,1001,NaT,5614,162,23,4699.26,n,error,0
2,1002,2016-01-13,5010,873,4,240.05,n,error,2
3,1003,NaT,5884,408,26,2754.57,n,error,0
4,1004,NaT,5015,774,19,1931.35,n,error,0


### Missing Values

Drop Rows with missing dates

In [23]:
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-01,-101,0,16,NaN,n,error,53
2,1002,2016-01-13,5010,873,4,240.05,n,error,2
6,1006,2016-01-16,5173,472,32,3445.96,n,error,2
8,1008,2016-01-17,5221,1017,24,16.25,n,error,2
10,1010,2016-06-01,5734,266,23,126.20,n,error,22


Fill Missing Price values with mean price

In [24]:
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

Fill Missing user_type values with value from previous row (forward fill) 

In [25]:
df_dropped['user_type'].fillna(method='ffill',inplace=True)

Fill Missing user_type values with value from next row (backward fill)

In [26]:
df_dropped['user_type'].fillna(method='bfill',inplace=True)

### Duplicates

Drop Duplicate serial_no rows

In [27]:
# sample duplicates
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
48,-1,2016-06-01,5922,828,5,3658.41,b,new,22
60,-1,2016-12-01,5372,257,7,2335.93,b,new,48
92,-1,2016-01-02,5542,1098,2,1887.98,b,new,53
95,-1,2016-01-22,5594,184,11,4399.92,a,new,3
178,-1,2016-01-19,5305,405,19,753.32,b,new,3


Shape of df=(969, 9)


In [28]:
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [29]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-01,-101,0,16,2283.99,n,error,53
2,1002,2016-01-13,5010,873,4,240.05,n,error,2
6,1006,2016-01-16,5173,472,32,3445.96,n,error,2
8,1008,2016-01-17,5221,1017,24,16.25,n,error,2
10,1010,2016-06-01,5734,266,23,126.20,n,error,22


Shape of df=(941, 9)


Remove rows which have less than 3 attributes with non-missing data

In [30]:
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week
0,1000,2016-01-01,-101,0,16,NaN,n,error,53
1,1001,NaT,5614,162,23,4699.26,n,error,0
2,1002,2016-01-13,5010,873,4,240.05,n,error,2
3,1003,NaT,5884,408,26,2754.57,n,error,0
4,1004,NaT,5015,774,19,1931.35,n,error,0


Shape of df=(1000, 9)


### Encode Categoricals

One Hot Encoding using get_dummies()

In [31]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,purchase_week,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,2016-01-01,-101,0,16,NaN,error,53,0,0,0,0,1
1,1001,NaT,5614,162,23,4699.26,error,0,0,0,0,0,1
2,1002,2016-01-13,5010,873,4,240.05,error,2,0,0,0,0,1
3,1003,NaT,5884,408,26,2754.57,error,0,0,0,0,0,1
4,1004,NaT,5015,774,19,1931.35,error,0,0,0,0,0,1


Label Mapping

In [32]:
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
995,1995,2016-07-02,5594,184,11,2538.75,b,new,26,1.0
996,1996,2016-01-24,5761,1031,14,791.87,b,new,3,1.0
997,1997,2016-01-26,5483,1086,7,435.93,a,new,4,0.0
998,1998,2016-01-23,5253,676,39,307.67,b,new,3,1.0
999,1999,2016-10-01,5362,654,24,NaN,d,loyal_existing,39,3.0


### Random Sampling data from DataFrame

In [33]:
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
102,1102,2016-05-02,5010,873,9,73.02,c,existing,18,2.0
435,1435,2016-01-13,5994,191,18,724.02,d,loyal_existing,2,3.0
860,1860,2016-01-15,5372,257,19,1484.65,c,existing,2,2.0
270,1270,2016-01-14,5785,710,13,2893.63,b,new,2,1.0
106,1106,2016-01-23,5173,472,23,NaN,a,new,3,0.0


### Normalizing Numeric Values

Normalize price values using  **Min-Max Scaler**

In [34]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [35]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
19,1019,2016-01-17,5303,970,14,0.411124,b,new,2,1.0
25,1025,2016-01-13,5808,509,1,0.305672,b,new,2,1.0
28,1028,2016-04-02,5574,188,28,0.081696,b,new,13,1.0
33,1033,2016-01-29,5289,191,24,0.316099,a,new,4,0.0
34,1034,2016-01-01,5223,1047,27,0.477160,a,new,53,0.0


Normalize quantity purchased values using  **Robust Scaler**

In [36]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [37]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,purchase_week,encoded_user_type
19,1019,2016-01-17,5303,970,-0.333333,3554.08,b,new,2,1.0
25,1025,2016-01-13,5808,509,-0.952381,2643.40,b,new,2,1.0
28,1028,2016-04-02,5574,188,0.333333,709.16,b,new,13,1.0
33,1033,2016-01-29,5289,191,0.142857,2733.45,a,new,4,0.0
34,1034,2016-01-01,5223,1047,0.285714,4124.36,a,new,53,0.0


### Data Summarization

Condition based aggregation

In [38]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2469.2060714285717


Condtion based counts

In [39]:
print(df['purchase_week'].value_counts())

4     165
3     163
2     117
35     53
26     53
22     49
44     47
31     47
13     45
39     41
9      39
5      37
53     36
0      31
17     26
48     26
18     25
Name: purchase_week, dtype: int64


### Group By

Group By certain attributes

In [40]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error               661
existing           4399
loyal_existing     5654
new               10250
Name: quantity_purchased, dtype: int32


Group By with different aggregate functions

In [41]:
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,661,21.322581,31
existing,4399,19.904977,221
loyal_existing,5654,22.085938,256
new,10250,20.833333,492


Group by specific aggregate functions for each attribute

In [42]:
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2183.803448                  36
existing       c          2237.275093                  41
loyal_existing d          2133.766129                  41
new            a          2469.206071                  41
               b          2317.998471                  41

Group by with multiple agg for each attribute

In [43]:
display(df.groupby(['user_class','user_type']).agg({'price':{
                                                                'total_price':np.sum,
                                                                'mean_price':np.mean,
                                                                'variance_price':np.std,
                                                                'count':np.count_nonzero},
                                                   'quantity_purchased':np.sum}))  

d:\users\703177402\appdata\local\continuum\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\groupby\generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


price                                     \
                         total_price   mean_price variance_price  count   
user_class     user_type                                                  
error          n            63330.30  2183.803448    1458.418194   31.0   
existing       c           478776.87  2237.275093    1613.834495  221.0   
loyal_existing d           529174.00  2133.766129    1495.806675  256.0   
new            a           553102.16  2469.206071    1634.340743  232.0   
               b           591089.61  2317.998471    1613.345716  260.0   

                         quantity_purchased  
                                        sum  
user_class     user_type                     
error          n                        661  
existing       c                       4399  
loyal_existing d                       5654  
new            a                       4683  
               b                       5567

### Pivot Tables

In [44]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,2715.755000,3847.827500,1593.358000,1991.310000,NaN
2016-01-02,2859.911429,2098.911429,2084.663333,1017.080000,NaN
2016-01-13,1965.486000,1744.517500,2064.024000,1437.346000,1085.473333
2016-01-14,2927.310000,3098.835000,2056.733333,2055.202500,NaN
2016-01-15,2331.312500,2765.250000,3007.260000,2171.784286,NaN
2016-01-16,2007.526667,1375.253333,2191.518750,2117.665000,3445.960000
2016-01-17,2679.750000,3801.930909,3247.020000,2595.114286,16.250000
2016-01-18,2824.843000,70.890000,1548.280000,2681.764286,NaN
2016-01-19,2283.893333,2572.635000,3390.767500,1241.282500,3334.150000


### Stack a Dataframe

In [45]:
print(df.stack())

0    serial_no                            1000
     date                  2016-01-01 00:00:00
     user_id                              -101
     product_id                              0
     quantity_purchased                     16
                                  ...         
999  quantity_purchased                     24
     user_type                               d
     user_class                 loyal_existing
     purchase_week                          39
     encoded_user_type                       3
Length: 9908, dtype: object
